# 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                 
    results = model.process(image)                
    image.flags.writeable = True                  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [19]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection

In [34]:
mylist = os.listdir("ISL Continents and Countries/")
# print(mylist)
ls = []
for x in mylist:
    ls.append(os.path.splitext(x)[0])

In [35]:
ls

['Afghanistan',
 'Africa (sign 1)',
 'Africa (sign 2)',
 'Antarctica',
 'Argentina',
 'Armenia',
 'Cambodia',
 'Canada',
 'Chile',
 'China',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Egypt (sign 1)',
 'Egypt (sign 2)',
 'El Salvador',
 'England (sign 1)',
 'Kazakhstan',
 'Kenya',
 'Korea',
 'Kuwait',
 'Maharashtra',
 'Mali']

In [24]:
DATA_PATH = os.path.join('MP_Data') 
actions = np.array(ls)
no_sequences = 30
sequence_length = 30

In [36]:
DATA_PATH = os.path.join('Pre') 

In [37]:
import shutil
shutil.rmtree(DATA_PATH)

In [38]:
DATA_PATH = os.path.join('Pre') 
actions = np.array(ls)
no_sequences = 1
sequence_length = 30

In [39]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [41]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        s="ISL Continents and Countries/"
        s = s+ action + '.mp4'
        cap = cv2.VideoCapture(s)
        for sequence in range(no_sequences):
            frame_num = 0
            while (True):
                ret, frame = cap.read()
                if ret == True:
                    image, results = mediapipe_detection(frame, holistic)
                    if not (results.left_hand_landmarks or results.right_hand_landmarks):
                        continue
                    draw_styled_landmarks(image, results)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                    np.save(npy_path, keypoints)
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break     
                    frame_num+=1
                else:
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [30]:
cap.release()
cv2.destroyAllWindows()

In [42]:
DATA_PATH = os.path.join('MP_Data') 

In [43]:
shutil.rmtree(DATA_PATH)

In [44]:
DATA_PATH = os.path.join('MP_Data') 
actions = np.array(ls)
no_sequences = 30
sequence_length = 30

In [45]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [46]:
import shutil
for action in actions:
    path="Pre/"
    dpath="MP_Data/"
    path+=action+"/0/"
    dpath+=action+"/"
    prepointer=0
    newspointer=0
    newsframe=0
    preend=len(os.listdir(path))
    while(True):
        if(prepointer==preend):
            prepointer=0
        if(newsframe==sequence_length):
            newsframe=0
            newspointer+=1
        if(newspointer==no_sequences):
            break
        src = path+str(prepointer)+'.npy'
        
        dst =  dpath+str(newspointer)+'/'+str(newsframe)+'.npy'
        print(dst)
        t =shutil.copyfile(src, dst)
        prepointer+=1
        newsframe+=1
    

MP_Data/Afghanistan/0/0.npy
MP_Data/Afghanistan/0/1.npy
MP_Data/Afghanistan/0/2.npy
MP_Data/Afghanistan/0/3.npy
MP_Data/Afghanistan/0/4.npy
MP_Data/Afghanistan/0/5.npy
MP_Data/Afghanistan/0/6.npy
MP_Data/Afghanistan/0/7.npy
MP_Data/Afghanistan/0/8.npy
MP_Data/Afghanistan/0/9.npy
MP_Data/Afghanistan/0/10.npy
MP_Data/Afghanistan/0/11.npy
MP_Data/Afghanistan/0/12.npy
MP_Data/Afghanistan/0/13.npy
MP_Data/Afghanistan/0/14.npy
MP_Data/Afghanistan/0/15.npy
MP_Data/Afghanistan/0/16.npy
MP_Data/Afghanistan/0/17.npy
MP_Data/Afghanistan/0/18.npy
MP_Data/Afghanistan/0/19.npy
MP_Data/Afghanistan/0/20.npy
MP_Data/Afghanistan/0/21.npy
MP_Data/Afghanistan/0/22.npy
MP_Data/Afghanistan/0/23.npy
MP_Data/Afghanistan/0/24.npy
MP_Data/Afghanistan/0/25.npy
MP_Data/Afghanistan/0/26.npy
MP_Data/Afghanistan/0/27.npy
MP_Data/Afghanistan/0/28.npy
MP_Data/Afghanistan/0/29.npy
MP_Data/Afghanistan/1/0.npy
MP_Data/Afghanistan/1/1.npy
MP_Data/Afghanistan/1/2.npy
MP_Data/Afghanistan/1/3.npy
MP_Data/Afghanistan/1/4.npy


MP_Data/Afghanistan/15/11.npy
MP_Data/Afghanistan/15/12.npy
MP_Data/Afghanistan/15/13.npy
MP_Data/Afghanistan/15/14.npy
MP_Data/Afghanistan/15/15.npy
MP_Data/Afghanistan/15/16.npy
MP_Data/Afghanistan/15/17.npy
MP_Data/Afghanistan/15/18.npy
MP_Data/Afghanistan/15/19.npy
MP_Data/Afghanistan/15/20.npy
MP_Data/Afghanistan/15/21.npy
MP_Data/Afghanistan/15/22.npy
MP_Data/Afghanistan/15/23.npy
MP_Data/Afghanistan/15/24.npy
MP_Data/Afghanistan/15/25.npy
MP_Data/Afghanistan/15/26.npy
MP_Data/Afghanistan/15/27.npy
MP_Data/Afghanistan/15/28.npy
MP_Data/Afghanistan/15/29.npy
MP_Data/Afghanistan/16/0.npy
MP_Data/Afghanistan/16/1.npy
MP_Data/Afghanistan/16/2.npy
MP_Data/Afghanistan/16/3.npy
MP_Data/Afghanistan/16/4.npy
MP_Data/Afghanistan/16/5.npy
MP_Data/Afghanistan/16/6.npy
MP_Data/Afghanistan/16/7.npy
MP_Data/Afghanistan/16/8.npy
MP_Data/Afghanistan/16/9.npy
MP_Data/Afghanistan/16/10.npy
MP_Data/Afghanistan/16/11.npy
MP_Data/Afghanistan/16/12.npy
MP_Data/Afghanistan/16/13.npy
MP_Data/Afghanistan/

MP_Data/Africa (sign 1)/0/9.npy
MP_Data/Africa (sign 1)/0/10.npy
MP_Data/Africa (sign 1)/0/11.npy
MP_Data/Africa (sign 1)/0/12.npy
MP_Data/Africa (sign 1)/0/13.npy
MP_Data/Africa (sign 1)/0/14.npy
MP_Data/Africa (sign 1)/0/15.npy
MP_Data/Africa (sign 1)/0/16.npy
MP_Data/Africa (sign 1)/0/17.npy
MP_Data/Africa (sign 1)/0/18.npy
MP_Data/Africa (sign 1)/0/19.npy
MP_Data/Africa (sign 1)/0/20.npy
MP_Data/Africa (sign 1)/0/21.npy
MP_Data/Africa (sign 1)/0/22.npy
MP_Data/Africa (sign 1)/0/23.npy
MP_Data/Africa (sign 1)/0/24.npy
MP_Data/Africa (sign 1)/0/25.npy
MP_Data/Africa (sign 1)/0/26.npy
MP_Data/Africa (sign 1)/0/27.npy
MP_Data/Africa (sign 1)/0/28.npy
MP_Data/Africa (sign 1)/0/29.npy
MP_Data/Africa (sign 1)/1/0.npy
MP_Data/Africa (sign 1)/1/1.npy
MP_Data/Africa (sign 1)/1/2.npy
MP_Data/Africa (sign 1)/1/3.npy
MP_Data/Africa (sign 1)/1/4.npy
MP_Data/Africa (sign 1)/1/5.npy
MP_Data/Africa (sign 1)/1/6.npy
MP_Data/Africa (sign 1)/1/7.npy
MP_Data/Africa (sign 1)/1/8.npy
MP_Data/Africa (sign

MP_Data/Africa (sign 1)/14/4.npy
MP_Data/Africa (sign 1)/14/5.npy
MP_Data/Africa (sign 1)/14/6.npy
MP_Data/Africa (sign 1)/14/7.npy
MP_Data/Africa (sign 1)/14/8.npy
MP_Data/Africa (sign 1)/14/9.npy
MP_Data/Africa (sign 1)/14/10.npy
MP_Data/Africa (sign 1)/14/11.npy
MP_Data/Africa (sign 1)/14/12.npy
MP_Data/Africa (sign 1)/14/13.npy
MP_Data/Africa (sign 1)/14/14.npy
MP_Data/Africa (sign 1)/14/15.npy
MP_Data/Africa (sign 1)/14/16.npy
MP_Data/Africa (sign 1)/14/17.npy
MP_Data/Africa (sign 1)/14/18.npy
MP_Data/Africa (sign 1)/14/19.npy
MP_Data/Africa (sign 1)/14/20.npy
MP_Data/Africa (sign 1)/14/21.npy
MP_Data/Africa (sign 1)/14/22.npy
MP_Data/Africa (sign 1)/14/23.npy
MP_Data/Africa (sign 1)/14/24.npy
MP_Data/Africa (sign 1)/14/25.npy
MP_Data/Africa (sign 1)/14/26.npy
MP_Data/Africa (sign 1)/14/27.npy
MP_Data/Africa (sign 1)/14/28.npy
MP_Data/Africa (sign 1)/14/29.npy
MP_Data/Africa (sign 1)/15/0.npy
MP_Data/Africa (sign 1)/15/1.npy
MP_Data/Africa (sign 1)/15/2.npy
MP_Data/Africa (sign 1)

MP_Data/Africa (sign 1)/27/15.npy
MP_Data/Africa (sign 1)/27/16.npy
MP_Data/Africa (sign 1)/27/17.npy
MP_Data/Africa (sign 1)/27/18.npy
MP_Data/Africa (sign 1)/27/19.npy
MP_Data/Africa (sign 1)/27/20.npy
MP_Data/Africa (sign 1)/27/21.npy
MP_Data/Africa (sign 1)/27/22.npy
MP_Data/Africa (sign 1)/27/23.npy
MP_Data/Africa (sign 1)/27/24.npy
MP_Data/Africa (sign 1)/27/25.npy
MP_Data/Africa (sign 1)/27/26.npy
MP_Data/Africa (sign 1)/27/27.npy
MP_Data/Africa (sign 1)/27/28.npy
MP_Data/Africa (sign 1)/27/29.npy
MP_Data/Africa (sign 1)/28/0.npy
MP_Data/Africa (sign 1)/28/1.npy
MP_Data/Africa (sign 1)/28/2.npy
MP_Data/Africa (sign 1)/28/3.npy
MP_Data/Africa (sign 1)/28/4.npy
MP_Data/Africa (sign 1)/28/5.npy
MP_Data/Africa (sign 1)/28/6.npy
MP_Data/Africa (sign 1)/28/7.npy
MP_Data/Africa (sign 1)/28/8.npy
MP_Data/Africa (sign 1)/28/9.npy
MP_Data/Africa (sign 1)/28/10.npy
MP_Data/Africa (sign 1)/28/11.npy
MP_Data/Africa (sign 1)/28/12.npy
MP_Data/Africa (sign 1)/28/13.npy
MP_Data/Africa (sign 1)/

...

MP_Data/Kuwait/20/12.npy
MP_Data/Kuwait/20/13.npy
MP_Data/Kuwait/20/14.npy
MP_Data/Kuwait/20/15.npy
MP_Data/Kuwait/20/16.npy
MP_Data/Kuwait/20/17.npy
MP_Data/Kuwait/20/18.npy
MP_Data/Kuwait/20/19.npy
MP_Data/Kuwait/20/20.npy
MP_Data/Kuwait/20/21.npy
MP_Data/Kuwait/20/22.npy
MP_Data/Kuwait/20/23.npy
MP_Data/Kuwait/20/24.npy
MP_Data/Kuwait/20/25.npy
MP_Data/Kuwait/20/26.npy
MP_Data/Kuwait/20/27.npy
MP_Data/Kuwait/20/28.npy
MP_Data/Kuwait/20/29.npy
MP_Data/Kuwait/21/0.npy
MP_Data/Kuwait/21/1.npy
MP_Data/Kuwait/21/2.npy
MP_Data/Kuwait/21/3.npy
MP_Data/Kuwait/21/4.npy
MP_Data/Kuwait/21/5.npy
MP_Data/Kuwait/21/6.npy
MP_Data/Kuwait/21/7.npy
MP_Data/Kuwait/21/8.npy
MP_Data/Kuwait/21/9.npy
MP_Data/Kuwait/21/10.npy
MP_Data/Kuwait/21/11.npy
MP_Data/Kuwait/21/12.npy
MP_Data/Kuwait/21/13.npy
MP_Data/Kuwait/21/14.npy
MP_Data/Kuwait/21/15.npy
MP_Data/Kuwait/21/16.npy
MP_Data/Kuwait/21/17.npy
MP_Data/Kuwait/21/18.npy
MP_Data/Kuwait/21/19.npy
MP_Data/Kuwait/21/20.npy
MP_Data/Kuwait/21/21.npy
MP_Data/Ku

MP_Data/Maharashtra/7/26.npy
MP_Data/Maharashtra/7/27.npy
MP_Data/Maharashtra/7/28.npy
MP_Data/Maharashtra/7/29.npy
MP_Data/Maharashtra/8/0.npy
MP_Data/Maharashtra/8/1.npy
MP_Data/Maharashtra/8/2.npy
MP_Data/Maharashtra/8/3.npy
MP_Data/Maharashtra/8/4.npy
MP_Data/Maharashtra/8/5.npy
MP_Data/Maharashtra/8/6.npy
MP_Data/Maharashtra/8/7.npy
MP_Data/Maharashtra/8/8.npy
MP_Data/Maharashtra/8/9.npy
MP_Data/Maharashtra/8/10.npy
MP_Data/Maharashtra/8/11.npy
MP_Data/Maharashtra/8/12.npy
MP_Data/Maharashtra/8/13.npy
MP_Data/Maharashtra/8/14.npy
MP_Data/Maharashtra/8/15.npy
MP_Data/Maharashtra/8/16.npy
MP_Data/Maharashtra/8/17.npy
MP_Data/Maharashtra/8/18.npy
MP_Data/Maharashtra/8/19.npy
MP_Data/Maharashtra/8/20.npy
MP_Data/Maharashtra/8/21.npy
MP_Data/Maharashtra/8/22.npy
MP_Data/Maharashtra/8/23.npy
MP_Data/Maharashtra/8/24.npy
MP_Data/Maharashtra/8/25.npy
MP_Data/Maharashtra/8/26.npy
MP_Data/Maharashtra/8/27.npy
MP_Data/Maharashtra/8/28.npy
MP_Data/Maharashtra/8/29.npy
MP_Data/Maharashtra/9/0.

MP_Data/Maharashtra/22/8.npy
MP_Data/Maharashtra/22/9.npy
MP_Data/Maharashtra/22/10.npy
MP_Data/Maharashtra/22/11.npy
MP_Data/Maharashtra/22/12.npy
MP_Data/Maharashtra/22/13.npy
MP_Data/Maharashtra/22/14.npy
MP_Data/Maharashtra/22/15.npy
MP_Data/Maharashtra/22/16.npy
MP_Data/Maharashtra/22/17.npy
MP_Data/Maharashtra/22/18.npy
MP_Data/Maharashtra/22/19.npy
MP_Data/Maharashtra/22/20.npy
MP_Data/Maharashtra/22/21.npy
MP_Data/Maharashtra/22/22.npy
MP_Data/Maharashtra/22/23.npy
MP_Data/Maharashtra/22/24.npy
MP_Data/Maharashtra/22/25.npy
MP_Data/Maharashtra/22/26.npy
MP_Data/Maharashtra/22/27.npy
MP_Data/Maharashtra/22/28.npy
MP_Data/Maharashtra/22/29.npy
MP_Data/Maharashtra/23/0.npy
MP_Data/Maharashtra/23/1.npy
MP_Data/Maharashtra/23/2.npy
MP_Data/Maharashtra/23/3.npy
MP_Data/Maharashtra/23/4.npy
MP_Data/Maharashtra/23/5.npy
MP_Data/Maharashtra/23/6.npy
MP_Data/Maharashtra/23/7.npy
MP_Data/Maharashtra/23/8.npy
MP_Data/Maharashtra/23/9.npy
MP_Data/Maharashtra/23/10.npy
MP_Data/Maharashtra/23

MP_Data/Mali/9/16.npy
MP_Data/Mali/9/17.npy
MP_Data/Mali/9/18.npy
MP_Data/Mali/9/19.npy
MP_Data/Mali/9/20.npy
MP_Data/Mali/9/21.npy
MP_Data/Mali/9/22.npy
MP_Data/Mali/9/23.npy
MP_Data/Mali/9/24.npy
MP_Data/Mali/9/25.npy
MP_Data/Mali/9/26.npy
MP_Data/Mali/9/27.npy
MP_Data/Mali/9/28.npy
MP_Data/Mali/9/29.npy
MP_Data/Mali/10/0.npy
MP_Data/Mali/10/1.npy
MP_Data/Mali/10/2.npy
MP_Data/Mali/10/3.npy
MP_Data/Mali/10/4.npy
MP_Data/Mali/10/5.npy
MP_Data/Mali/10/6.npy
MP_Data/Mali/10/7.npy
MP_Data/Mali/10/8.npy
MP_Data/Mali/10/9.npy
MP_Data/Mali/10/10.npy
MP_Data/Mali/10/11.npy
MP_Data/Mali/10/12.npy
MP_Data/Mali/10/13.npy
MP_Data/Mali/10/14.npy
MP_Data/Mali/10/15.npy
MP_Data/Mali/10/16.npy
MP_Data/Mali/10/17.npy
MP_Data/Mali/10/18.npy
MP_Data/Mali/10/19.npy
MP_Data/Mali/10/20.npy
MP_Data/Mali/10/21.npy
MP_Data/Mali/10/22.npy
MP_Data/Mali/10/23.npy
MP_Data/Mali/10/24.npy
MP_Data/Mali/10/25.npy
MP_Data/Mali/10/26.npy
MP_Data/Mali/10/27.npy
MP_Data/Mali/10/28.npy
MP_Data/Mali/10/29.npy
MP_Data/Mali

MP_Data/Mali/24/9.npy
MP_Data/Mali/24/10.npy
MP_Data/Mali/24/11.npy
MP_Data/Mali/24/12.npy
MP_Data/Mali/24/13.npy
MP_Data/Mali/24/14.npy
MP_Data/Mali/24/15.npy
MP_Data/Mali/24/16.npy
MP_Data/Mali/24/17.npy
MP_Data/Mali/24/18.npy
MP_Data/Mali/24/19.npy
MP_Data/Mali/24/20.npy
MP_Data/Mali/24/21.npy
MP_Data/Mali/24/22.npy
MP_Data/Mali/24/23.npy
MP_Data/Mali/24/24.npy
MP_Data/Mali/24/25.npy
MP_Data/Mali/24/26.npy
MP_Data/Mali/24/27.npy
MP_Data/Mali/24/28.npy
MP_Data/Mali/24/29.npy
MP_Data/Mali/25/0.npy
MP_Data/Mali/25/1.npy
MP_Data/Mali/25/2.npy
MP_Data/Mali/25/3.npy
MP_Data/Mali/25/4.npy
MP_Data/Mali/25/5.npy
MP_Data/Mali/25/6.npy
MP_Data/Mali/25/7.npy
MP_Data/Mali/25/8.npy
MP_Data/Mali/25/9.npy
MP_Data/Mali/25/10.npy
MP_Data/Mali/25/11.npy
MP_Data/Mali/25/12.npy
MP_Data/Mali/25/13.npy
MP_Data/Mali/25/14.npy
MP_Data/Mali/25/15.npy
MP_Data/Mali/25/16.npy
MP_Data/Mali/25/17.npy
MP_Data/Mali/25/18.npy
MP_Data/Mali/25/19.npy
MP_Data/Mali/25/20.npy
MP_Data/Mali/25/21.npy
MP_Data/Mali/25/22.npy

# 6. Preprocess Data and Create Labels and Features

In [47]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [48]:
label_map = {label:num for num, label in enumerate(actions)}

In [49]:
label_map

{'Afghanistan': 0,
 'Africa (sign 1)': 1,
 'Africa (sign 2)': 2,
 'Antarctica': 3,
 'Argentina': 4,
 'Armenia': 5,
 'Cambodia': 6,
 'Canada': 7,
 'Chile': 8,
 'China': 9,
 'Costa Rica': 10,
 'Croatia': 11,
 'Cyprus': 12,
 'Czech Republic': 13,
 'Denmark': 14,
 'Egypt (sign 1)': 15,
 'Egypt (sign 2)': 16,
 'El Salvador': 17,
 'England (sign 1)': 18,
 'Kazakhstan': 19,
 'Kenya': 20,
 'Korea': 21,
 'Kuwait': 22,
 'Maharashtra': 23,
 'Mali': 24}

In [50]:
sequences, labels = [], []
lis = []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
            lis.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [51]:
np.array(sequences).shape

(750, 30, 1662)

In [53]:
X = np.array(sequences)

In [54]:
X.shape

(750, 30, 1662)

In [55]:
y = to_categorical(labels).astype(int)

In [56]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [58]:
y_test.shape

(38, 25)

# 7. Build and Train LSTM Neural Network

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [60]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [61]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [62]:
res = [.7, 0.2, 0.1]

In [63]:
actions[np.argmax(res)]

'Afghanistan'

In [64]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [66]:
model.fit(X_train, y_train, epochs=500,batch_size=100, callbacks=[tb_callback])

Epoch 1/500
8/8 [==============================] - 5s 200ms/step - loss: 3.2578 - categorical_accuracy: 0.0379
Epoch 2/500
8/8 [==============================] - 1s 143ms/step - loss: 3.5827 - categorical_accuracy: 0.0562
Epoch 3/500
8/8 [==============================] - 1s 155ms/step - loss: 3.2318 - categorical_accuracy: 0.0337
Epoch 4/500
8/8 [==============================] - 1s 143ms/step - loss: 3.2165 - categorical_accuracy: 0.0365
Epoch 5/500
8/8 [==============================] - 1s 143ms/step - loss: 3.2248 - categorical_accuracy: 0.0478
Epoch 6/500
8/8 [==============================] - 1s 151ms/step - loss: 3.2041 - categorical_accuracy: 0.0801
Epoch 7/500
8/8 [==============================] - 2s 205ms/step - loss: 3.2255 - categorical_accuracy: 0.0435
Epoch 8/500
8/8 [==============================] - 2s 205ms/step - loss: 3.2135 - categorical_accuracy: 0.0590
Epoch 9/500
8/8 [==============================] - 2s 188ms/step - loss: 3.2193 - categorical_accuracy: 0.0435
E

8/8 [==============================] - 2s 205ms/step - loss: 1.1251 - categorical_accuracy: 0.5267
Epoch 146/500
8/8 [==============================] - 2s 186ms/step - loss: 1.0246 - categorical_accuracy: 0.5787
Epoch 147/500
8/8 [==============================] - 1s 181ms/step - loss: 1.0310 - categorical_accuracy: 0.5520
Epoch 148/500
8/8 [==============================] - 1s 174ms/step - loss: 1.1205 - categorical_accuracy: 0.4874
Epoch 149/500
8/8 [==============================] - 1s 174ms/step - loss: 1.1337 - categorical_accuracy: 0.4986
Epoch 150/500
8/8 [==============================] - 1s 174ms/step - loss: 0.9456 - categorical_accuracy: 0.5871
Epoch 151/500
8/8 [==============================] - 1s 178ms/step - loss: 0.9111 - categorical_accuracy: 0.6138
Epoch 152/500
8/8 [==============================] - 1s 174ms/step - loss: 0.8354 - categorical_accuracy: 0.6573
Epoch 153/500
8/8 [==============================] - 1s 174ms/step - loss: 0.8241 - categorical_accuracy: 0.62

8/8 [==============================] - 1s 174ms/step - loss: 0.3859 - categorical_accuracy: 0.8287
Epoch 289/500
8/8 [==============================] - 1s 176ms/step - loss: 0.3834 - categorical_accuracy: 0.8469
Epoch 290/500
8/8 [==============================] - 1s 172ms/step - loss: 0.3984 - categorical_accuracy: 0.8216
Epoch 291/500
8/8 [==============================] - 1s 174ms/step - loss: 0.4166 - categorical_accuracy: 0.8020
Epoch 292/500
8/8 [==============================] - 1s 174ms/step - loss: 0.6350 - categorical_accuracy: 0.7247
Epoch 293/500
8/8 [==============================] - 1s 176ms/step - loss: 1.1002 - categorical_accuracy: 0.58710s - loss: 1.2687 - categorical_accura
Epoch 294/500
8/8 [==============================] - 1s 176ms/step - loss: 1.1262 - categorical_accuracy: 0.5927
Epoch 295/500
8/8 [==============================] - 1s 176ms/step - loss: 1.2133 - categorical_accuracy: 0.5435
Epoch 296/500
8/8 [==============================] - 1s 176ms/step - los

8/8 [==============================] - 1s 178ms/step - loss: 0.1859 - categorical_accuracy: 0.9508
Epoch 432/500
8/8 [==============================] - 1s 174ms/step - loss: 0.1491 - categorical_accuracy: 0.9817
Epoch 433/500
8/8 [==============================] - 1s 176ms/step - loss: 0.1389 - categorical_accuracy: 0.9705
Epoch 434/500
8/8 [==============================] - 1s 174ms/step - loss: 0.1391 - categorical_accuracy: 0.9761
Epoch 435/500
8/8 [==============================] - 1s 176ms/step - loss: 0.1215 - categorical_accuracy: 0.9649
Epoch 436/500
8/8 [==============================] - 1s 176ms/step - loss: 0.0797 - categorical_accuracy: 0.9831
Epoch 437/500
8/8 [==============================] - 2s 189ms/step - loss: 0.0752 - categorical_accuracy: 0.9902
Epoch 438/500
8/8 [==============================] - 1s 179ms/step - loss: 0.0708 - categorical_accuracy: 0.9831
Epoch 439/500
8/8 [==============================] - 1s 174ms/step - loss: 0.0592 - categorical_accuracy: 0.98

In [67]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 64)            33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 25)                8

# 8. Make Predictions

In [68]:
res = model.predict(X_test)

In [69]:
actions[np.argmax(res[4])]

'Maharashtra'

In [70]:
actions[np.argmax(y_test[4])]

'Maharashtra'

# 9. Save Weights

In [71]:
model.save('midsem.h5')

In [ ]:
del model

In [13]:
from tensorflow import keras
model = keras.models.load_model('action.h5')

In [12]:
model.load_weights('action.h5')

NameError: name 'model' is not defined

# 10. Evaluation using Confusion Matrix and Accuracy

In [72]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [73]:
yhat = model.predict(X_test)

In [74]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [75]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[36,  1],
        [ 0,  1]],

       [[34,  0],
        [ 0,  4]],

       [[37,  0],
        [ 0,  1]],

       [[36,  0],
        [ 0,  2]],

       [[37,  0],
        [ 0,  1]],

       [[37,  0],
        [ 0,  1]],

       [[36,  0],
        [ 0,  2]],

       [[34,  0],
        [ 1,  3]],

       [[35,  0],
        [ 0,  3]],

       [[36,  0],
        [ 0,  2]],

       [[36,  1],
        [ 0,  1]],

       [[36,  0],
        [ 1,  1]],

       [[37,  0],
        [ 0,  1]],

       [[35,  0],
        [ 0,  3]],

       [[35,  0],
        [ 0,  3]],

       [[37,  0],
        [ 1,  0]],

       [[36,  1],
        [ 0,  1]],

       [[36,  0],
        [ 0,  2]],

       [[36,  0],
        [ 0,  2]],

       [[37,  0],
        [ 0,  1]]], dtype=int64)

In [77]:
accuracy_score(ytrue, yhat)

0.9210526315789473

# 11. Test in Real Time

In [85]:
colors = []
for i in actions:
    colors.append((245,117,16))
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    return output_frame

In [91]:

import math
sequence = []
sentence = []
predictions = []
threshold = 0.3
frameRate = cap.get(5)
#ISL Continents and Countries/Mali.mp4
cap = cv2.VideoCapture('ISL Continents and Countries/Croatia.mp4')
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if(ret==True):
            image, results = mediapipe_detection(frame, holistic)
            if not (results.left_hand_landmarks or results.right_hand_landmarks):
                continue
            draw_styled_landmarks(image, results)
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
                predictions.append(np.argmax(res))
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
                if len(sentence) > 3: 
                    sentence = sentence[-3:]
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            i=0
            while(len(sequence)<30):
                sequence.append(sequence[i])
                i+=1
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                predictions.append(np.argmax(res))
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 
                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 30: 
                    sentence = sentence[-3:]
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

In [88]:
cap.release()
cv2.destroyAllWindows()